In [21]:
import torch
import os
from pandas import read_csv
from torch.optim import SGD
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from config import DATA_DIR, MODELS_DIR
from src.loss import Loss
from src.model import HistogramMF
from src.runner import Runner
from src.create_dataset import create_dataset, create_histogram_features
from src.data_processor import DataProcessor
from src.data_encoder import DataEncoder

DF_PATH = (
    f"{DATA_DIR}"
    f"/DEAM/annotations/annotations per each rater/"
    f"song_level/static_annotations_songs_1_2000_raw.csv"
)

In [22]:
columns = ["workerID", "SongId", "Valence", "Arousal"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "Valence", "Arousal"]
original_df.head()

,user_id,item_id,Valence,Arousal
0,6010bbc8e7ef4b21fa38f9c3a9754ef3,2,5,2
1,3c888e77b992ae3cd2adfe16774e23b9,2,2,3
2,2afd218c3aecb6828d2be327f8b9c46f,2,3,3
3,fd5b08ce362d855ca9152a894348130c,2,4,4
4,9c8073214a052e414811b76012df8847,2,2,2


In [23]:
valence_dataframe = original_df[["user_id", "item_id", "Valence"]].copy()
valence_dataframe.columns = ["user_id", "item_id", "rating"]
create_histogram_features(data_frame=valence_dataframe)

data_encoder = DataEncoder(original_df=valence_dataframe)
data_processor = DataProcessor(original_df=valence_dataframe)

n_users = valence_dataframe.user_id.nunique()
n_items = valence_dataframe.item_id.nunique()

min_rating = min(valence_dataframe.rating.values)
max_rating = max(valence_dataframe.rating.values)

valence_model = HistogramMF(
    n_users=n_users,
    n_items=n_items,
    data_encoder=data_encoder,
    data_processor=data_processor,
    min_rating=min_rating,
    max_rating=max_rating,
)

if os.path.exists(f"{MODELS_DIR}/DEAM/valence.pt"):
    valence_model.load_state_dict(torch.load(f"{MODELS_DIR}/DEAM/valence.pt"))
else:
    epochs = 100

    criterion = Loss()
    optimizer = SGD(valence_model.parameters(), lr=5, weight_decay=1e-7)
    runner = Runner(model=valence_model, criterion=criterion, optimizer=optimizer)

    train_set = create_dataset(data_encoder=data_encoder)
    train_load = DataLoader(train_set, batch_size=1000, shuffle=True)

    with SummaryWriter(f"runs/DEAM/dev/valence") as writer:
        for epoch in range(epochs):
            epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
            print(f"epoch={epoch + 1}, loss={epoch_loss}")

    torch.save(valence_model.state_dict(), f"{MODELS_DIR}/DEAM/valence.pt")

  0%|          | 0/17464 [00:00<?, ?it/s]

  0%|          | 0/17464 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:06<00:00,  2.86batch/s, train_loss=0.325]


epoch=1, loss=6.521853092851309


100%|██████████| 18/18 [00:04<00:00,  3.98batch/s, train_loss=0.267]


...

epoch=98, loss=0.021398619867090525


100%|██████████| 18/18 [00:04<00:00,  4.02batch/s, train_loss=0.00131] 


epoch=99, loss=0.02111411698213939


100%|██████████| 18/18 [00:04<00:00,  3.89batch/s, train_loss=0.00189] 

epoch=100, loss=0.021990848892721637


In [24]:
arousal_dataframe = original_df[["user_id", "item_id", "Arousal"]].copy()
arousal_dataframe.columns = ["user_id", "item_id", "rating"]
create_histogram_features(data_frame=arousal_dataframe)

data_encoder = DataEncoder(original_df=arousal_dataframe)
data_processor = DataProcessor(original_df=arousal_dataframe)

n_users = arousal_dataframe.user_id.nunique()
n_items = arousal_dataframe.item_id.nunique()

min_rating = min(arousal_dataframe.rating.values)
max_rating = max(arousal_dataframe.rating.values)

arousal_model = HistogramMF(
    n_users=n_users,
    n_items=n_items,
    data_encoder=data_encoder,
    data_processor=data_processor,
    min_rating=min_rating,
    max_rating=max_rating,
)

if os.path.exists(f"{MODELS_DIR}/DEAM/arousal.pt"):
    valence_model.load_state_dict(torch.load(f"{MODELS_DIR}/DEAM/arousal.pt"))
else:
    epochs = 100

    criterion = Loss()
    optimizer = SGD(arousal_model.parameters(), lr=5, weight_decay=1e-7)
    runner = Runner(model=arousal_model, criterion=criterion, optimizer=optimizer)

    train_set = create_dataset(data_encoder=data_encoder)
    train_load = DataLoader(train_set, batch_size=1000, shuffle=True)

    with SummaryWriter(f"runs/DEAM/dev/arousal") as writer:
        for epoch in range(epochs):
            epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
            print(f"epoch={epoch + 1}, loss={epoch_loss}")

    torch.save(arousal_model.state_dict(), f"{MODELS_DIR}/DEAM/arousal.pt")

  0%|          | 0/17464 [00:00<?, ?it/s]

  0%|          | 0/17464 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:05<00:00,  3.32batch/s, train_loss=0.317]


epoch=1, loss=6.653590763486664


100%|██████████| 18/18 [00:04<00:00,  3.68batch/s, train_loss=0.259]


...

epoch=98, loss=0.02004924507696053


100%|██████████| 18/18 [00:03<00:00,  5.51batch/s, train_loss=0.00189] 


epoch=99, loss=0.018742961042913894


100%|██████████| 18/18 [00:03<00:00,  5.39batch/s, train_loss=4.42e-5] 

epoch=100, loss=0.017887878675848762


In [25]:
from pandas import DataFrame
from collections import namedtuple

Row = namedtuple("Row", "user_id item_id Valence Arousal")

data_encoder = DataEncoder(original_df=original_df)
dataframe_after_mf = []

with torch.no_grad():
    for (index, user_id, item_id, valence, arousal) in original_df.itertuples():
        encoded_user_id = data_encoder.get_encoded_user_id(original_id=user_id)
        encoded_item_id = data_encoder.get_encoded_item_id(original_id=item_id)

        user_id_as_tensor = torch.LongTensor([encoded_user_id])
        item_id_as_tensor = torch.LongTensor([encoded_item_id])
        valence_output = valence_model(users=user_id_as_tensor, items=item_id_as_tensor,).squeeze()[0]
        arousal_output = arousal_model(users=user_id_as_tensor, items=item_id_as_tensor,).squeeze()[0]
        predicted_valence = torch.round(valence_output).item()
        predicted_arousal = torch.round(arousal_output).item()

        dataframe_after_mf.append(Row(user_id=user_id, item_id=item_id, Valence=predicted_valence, Arousal=predicted_arousal))

df_after_mf = DataFrame(dataframe_after_mf, columns=["user_id", "item_id", "Valence", "Arousal"])
df_after_mf.head()

,user_id,item_id,Valence,Arousal
0,6010bbc8e7ef4b21fa38f9c3a9754ef3,2,5.0,2.0
1,3c888e77b992ae3cd2adfe16774e23b9,2,2.0,3.0
2,2afd218c3aecb6828d2be327f8b9c46f,2,3.0,3.0
3,fd5b08ce362d855ca9152a894348130c,2,4.0,4.0
4,9c8073214a052e414811b76012df8847,2,2.0,2.0


In [26]:
mask = ((original_df["Valence"] == df_after_mf["Valence"]) & (original_df["Arousal"] == df_after_mf["Arousal"]))
changes = original_df[mask].copy()
changes["New Valence"] = df_after_mf.Valence
changes["New Arousal"] = df_after_mf.Arousal
print(f"Number of hits: {len(changes)} / {len(original_df)}")
print(f"Hit ratio: {(len(changes) / len(original_df)) * 100}")
changes.head(len(changes))

Number of hits: 17345 / 17464
Hit ratio: 99.31859825927623


,user_id,item_id,Valence,Arousal,New Valence,New Arousal
0,6010bbc8e7ef4b21fa38f9c3a9754ef3,2,5,2,5.0,2.0
1,3c888e77b992ae3cd2adfe16774e23b9,2,2,3,2.0,3.0
2,2afd218c3aecb6828d2be327f8b9c46f,2,3,3,3.0,3.0
3,fd5b08ce362d855ca9152a894348130c,2,4,4,4.0,4.0
4,9c8073214a052e414811b76012df8847,2,2,2,2.0,2.0
...,...,...,...,...,...,...
17459,607f6e34a0b5923333f6b16d3a59cc98,2000,6,5,6.0,5.0
17460,78b5e9744073532cc376976b5fc6b2fc,2000,7,7,7.0,7.0
17461,7cecbffe1da5ae974952db6c13695afe,2000,4,5,4.0,5.0
17462,ed7ed76453bd846859f5e6b9149df276,2000,6,7,6.0,7.0
